<a href="https://colab.research.google.com/github/th0void/KurtNab-Assessment/blob/main/Using_Margo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Importing notebooks as modules with Margo margin notes

This notebook demonstrates how to use [margo-loader](https://github.com/margo-notebooks/margo-loader-py) to import notebooks as code modules.

Unlike other libraries, Margo Loader uses a special syntax for preprocessor directives that let you control how code is exposed in a module representation of a notebook. If you open up greetings.ipynb (once we download it), you'll see some of these directives such as `ignore-cell` and `submodule` in action.

## Set up the environment

In this section we install the margo-loader dependency, import some requirements and download an example notebook [greetings.ipynb](https://github.com/margo-notebooks/margo-loader-py/blob/main/test_notebooks/greetings.ipynb) that we can use to demonstrate.



In [ ]:
!python --version

Python 3.9.16


In [1]:
# Install margo-loader from github repository
# !python -m pip install git+https://github.com/margo-notebooks/margo-loader-py --use-deprecated=backtrack-on-build-failures

In [3]:
!pip install margo-loader
!python -m pip install git+https://github.com/margo-notebooks/margo-loader-py.git

  Using cached margo_loader-1.0.4-py3-none-any.whl (13 kB)
  Using cached margo_parser-1.0.3-py3-none-any.whl (11 kB)
  Using cached lark-1.1.5-py3-none-any.whl (107 kB)
  Using cached PyYAML-5.3.1.tar.gz (269 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Cloning https://github.com/margo-notebooks/margo-loader-py.git to /tmp/pip-req-build-108u3i73
  Running command git clone --filter=blob:none --quiet https://github.com/margo-notebooks/margo-loader-py.git /tmp/pip-req-build-108u3i73
  Resolved https://github.com/margo-notebooks/margo-loader-py.git t

In [ ]:
import requests

## Download a sample notebook

Download a local copy of [greetings.ipynb](https://github.com/margo-notebooks/margo-loader-py/blob/main/test_notebooks/greetings.ipynb) into the colab runtime so we can import it.

In [ ]:
# Download a copy of a module noteboook
def download_demo_notebook():
  notebook_url = "https://raw.githubusercontent.com/margo-notebooks/margo-loader-py/main/test_notebooks/greetings.ipynb"
  open("./greetings.ipynb", "wb").write(requests.get(notebook_url).content)

download_demo_notebook()

## Import greetings.ipynb

 Now that we've installed and imported `margo_loader` and we've downloaded a copy of `greetings.ipynb` somewhere that we can import it, we can use Python's built-in `import` statement to load it, just as if the notebook were a `.py` file.

In [ ]:
import margo_loader
import greetings

## Exploring the module

The first cell in `greetings.ipynb` is a Markdown cell, which tells `margo_loader` to treat the cell as the documentation for the module (in Python this is stored in a special property called `__docstring__`. We can access the `__docstring__` on this or any other module by using the `help()` function.

In [ ]:
help(greetings)

Help on module greetings:

NAME
    greetings - # Greetings

DESCRIPTION
    This notebook demonstrates how to use Margo to annotate a notebook to be treated as a module by Margo Loader with virtual submodules. It defines two submodules, `grumpy` and `nice`, each of which have their own `say_hello()` function.
    
    This Notebook demonstrates the `ignore-cell` directive and the `submodule` assignment features of Margo Loader. 
    
    Also, everything in this cell is treated as the __docstring__ for the notebook when it is imported as a module!

SUBMODULES
    grumpy
    nice

FUNCTIONS
    say_hello(to='world')

FILE
    /content/greetings.ipynb




We can see from the `help()` output that greetings defines two submodules `grumpy` and nice. Let's use `help()` to find out more about those.

We can see from the cells below that they each define a `say_hello` function with the same function signature.

In [ ]:
help(greetings.grumpy)

Help on module greetings.grumpy in greetings:

NAME
    greetings.grumpy

FUNCTIONS
    say_hello(to='world')

FILE
    (built-in)




In [ ]:
help(greetings.nice)

Help on module greetings.nice in greetings:

NAME
    greetings.nice

FUNCTIONS
    say_hello(to='world')

FILE
    (built-in)




## Running the code

Now that we know there are two `say_hello` functions, let's run them.

In [ ]:
greetings.nice.say_hello("Jake")

In [ ]:
greetings.grumpy.say_hello("Jake")

'Oh, uhh, hi Jake...'

## Examining greetings.ipynb

In this section, we take a look at some of the special syntax that enabled [greetings.ipynb](https://github.com/margo-notebooks/margo-loader-py/blob/main/test_notebooks/greetings.ipynb) to be imported the way it was in this notebook.

### submodule assignment

The greetings notebook uses the Margo Loader `submodule` assignment to create virtual submodules within a the notebook's top-level module. This allows the notebook author to organize code cells. In this case it was handy to make two submodules that behaved the same but had different attitudes.

You'll see if you open up greetings.ipynb that it uses syntax like this to define a function that should reside in the `grumpy` submodule:

```python
# :: submodule: "grumpy" ::
def say_hello(to="world"):
    return f"Oh, uhh, hi {to}..."
```

### ignore-cell directive

You'll also see in greetings.ipynb that some cells begin with `# :: ignore-cell ::`. These cells are used to demonstrate usage of the `say_hello()` function, and we don't want to allow those cells to be imported when the notebook is treated as a module.